In [1]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta

sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [ ]:

### Function fire_timeline

def concat_subsets(files):
    df = []
    for f in files:
        manyfr = pd.read_csv(f)

        manyfr = gpd.GeoDataFrame(manyfr)

        manyfr.t = manyfr.t.astype("datetime64[ns]")
        df.append(manyfr)
    df = pd.concat(df)
    return(df)

def get_lt(lt_string = "Lt_CA_Quebec_"):
    files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/" + lt_string +"*.csv")
    return(concat_subsets(files))


def fire_timeline(fireID, 
                  lt,
                  year = 2023,
                  path_region="QuebecGlobalNRT_3571" , 
                  check_last = False, 
                  FWI_source = "station" ):
    
    '''
    '''
    
    ## Read in the largefire file of the fireID
    fr = fv.load_large_fire(fireID, year = year, path_region= path_region) ## Cluster of 2 fires. 

        ## Filter? 
            ## VIIRS Static source filter?
            ## WUI filter? 
    ## Subset lightning by time and space
    fr = fr.to_crs("3571")
    lt = gpd.GeoDataFrame(lt, geometry=gpd.points_from_xy(lt.lon, lt.lat), crs=4326) #4674
    lt = lt.to_crs("3571")
    
    
    min_threshold = fr.t.astype('datetime64[ns]').min() - timedelta(days = 10)
    possible_lt = lt[lt.t <= fr.t.min()]
    possible_lt = possible_lt[possible_lt.t >= min_threshold]

    oldest_perim = fr[fr.t == fr.t.max()]
    first_perim = fr[fr.t == fr.t.min()]
    first_perim.geometry = first_perim.buffer(750*2) ## Two viirs pixels???
    join_lt = gpd.sjoin(possible_lt, first_perim, op = 'within', how = "inner")
    
    ## Get distance to individuals ignitions
    # fr["perim_rank"] = fr.t.rank()
    # first_geom = fr[fr.perim_rank == 1].geometry
    # first_geom = first_geom.iloc[0]
    # num_starts = len(first_geom.geoms)
    # for i in range(0, num_starts):
    #     join_lt["dist_start_" + str(i)] = join_lt.distance(first_geom.geoms[i].centroid)
    #     print(fr[fr.perim_rank == 1].to_crs("4326").geometry.iloc[0].geoms[i].centroid)
        
    ## Rank candidate by distance
    # range_geoms = list(range(0, num_starts))
    # string = "dist_start_"
    # columns_dists = [string + str(x) for x in range_geoms]
    # top = len(join_lt) * 1 # Top 100%. Could cut to smaller range
    # dist_bool = join_lt[columns_dists].rank() <= top ## NEED a max distance cutoff. 
    # join_lt["candidate"] = dist_bool.any(axis = 1)
    
    ## Get raw VIIRS pixel timing
    date_string = fr.t.astype("datetime64[ns]").max().strftime("%Y%m%d%p")
    raw_obs_times = fv.raw_pixel_times(1462, date_string = date_string)
    raw_obs_times = raw_obs_times.reset_index()
    
    ## get station data
    if(source = "station"):
        st = pd.read_csv("s3://veda-data-store-staging/EIS/other/station-FWI/19900101.NRT/FWI/718270-99999.linear.HourlyFWIFromHourlyInterpContinuous.csv") ## Corrected record from Robert
        st.HH = st.HH.astype("int")
        st.YYYY = st.YYYY.astype("int")
        st.MM = st.MM.astype("int")
        st.DD = st.DD.astype("int")
        st = fv.date_convert(st)
        
    else:
        print("No other FWI extraction method ready. Sorry. ")
        break
    
    ## Do merging
    
    
        
    

        

    
